In [ ]:
using Pkg
Pkg.add(url="https://github.com/bergermann/Blackmage")

In [ ]:
using Blackmage, JLD2
# use julias help mode to read information of each function

In [ ]:
# connect to devices, insert respective IP and port number
#  MC -> motor control
# IDS -> Interferometric Displacement Sensor
# IDS web interface can be opened by putting the IP adress into any browser
device_mc = connect(ip"",0)     # motors
device_ids = connect(ip"",0)    # interferometers

In [ ]:
# run this to close connection
close(device_mc); close(device_ids)

WARNING: Emergency shutdown not yet properly tested/implemented. Discuss with Christoph what to do if anything goes wrong.

In [ ]:
# set all motors to external drive mode
# maxdist and target tolerance are given in IDS units
mcSetupFCM(device_mc;
    master=1,
    stepsize=100,
    tol=300,
    maxdist=5000,
    freqmaster=50,
    freqslave=70,
    temp=300)

In [ ]:
# stop all motors and set them back to direct drive mode
mcStopAll(device_mc)

In [ ]:
# set target distance for all three motors
# this needs to be done once ideally at the positions where interferometers were initialized
# distance can not go lower than 4_500_000 IDS units, therefore ideally initialize IDS
# as close to backplate as possilbe
mcTargetFCM(device_mc,0,:cm)

In [ ]:
# after initialization freely set target positions
mcTargetFCM(device_mc,1,:cm)

In [ ]:
# block program until target is reached
mcWaitForTarget(device_mc)

WARNING: Only proceed, if the following two cells execute successfully.

In [ ]:
# test if direct drive commands are still possible in flexdrive configuration
mcMove.(device_mc,[1,2,3],1,1)

In [ ]:
# test if motor control allows reduced step size
mcMove.(device_mc,[1,2,3],0,1; rss=90)

In [ ]:
# example procedure to test precision corrections

RSS = [100,90,80,70,60,50,40,30,20,10]  # relative step size values to test
nsteps = 10                             # move n steps at once
dir = 1                                 # direction to test for

d_m = zeros(3,2*length(RSS))            # repeatedly measure position after each step
d_s = zeros(3,2*length(RSS))            # with uncertainties

for i in eachindex(RSS)
    println("Iter $i")

    mcTargetFCM(device_mc,1,:cm)        # reset positions
    mcWaitForTarget(device_mc)
    sleep(1)

    d_m[:,2i-1], d_s[:,2i-1] = measurePos(device_ids,100)

    sleep(1)

    mcMove.(device_mc,[1,2,3],dir,nsteps)

    sleep(1)

    d_m[:,2i], d_s[:,2i] = measurePos(device_ids,100)

    sleep(1)
end

In [ ]:
# plot and safe plots and data

p1 = plot(RSS,(d_m[:,2:2:end]-d_m[:,1:2:ebd])'./1e12/1e-3; seriestype=:scatter,
    xlabel="RSS",ylabel="Δx [mm]")

P = [p1]

f = "data3"
for i in eachindex(P)
    if isfile("$f.jld2")
        @warn "Plot file already exists."
    else
        savefig(P[i],"plots/$(f)_$i.svg")
    end
end

if !isfile("$f.jld2"); @save f d d_DL d_m d_s; else; @warn "File already exists."; end
